In [25]:
# !pip install faiss-cpu langchain_community langchain_openai langchain_groq sentence-transformers

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate, PromptTemplate
import os

os.environ["GROQ_API_KEY"] = "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"
groq_api_key = "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"

In [4]:
loader = TextLoader("yandex_2023.txt")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [5]:
embeddings = HuggingFaceEmbeddings(model_name='intfloat/multilingual-e5-large')
db = FAISS.from_documents(docs, embeddings)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [6]:
answer_question_prompt = """Вы опытный финансовый аудитор, занимающийся анализом деятельности компаний.
Используйте представленный далее фрагмент контекста из отчета компании, чтобы ответить на следующий вопрос:
{question}

Контекст из отчета компании:
{context}

В ответе не добавляй от себя ничего, следуй строго следующей инструкции:
Если в контексте есть ответ на вопрос, то приведи все необходимые предложения или факты из контекста, подтверждающие ответ. Если ответа на вопрос в контексте нет или если информации недостаточно, то верни только "None".
Если вы не знаете ответ, тогда укажите только "None" в качестве ответа.
"""

PROMPT = PromptTemplate(template=answer_question_prompt,
                        input_variables=["context", "question"])

llm = ChatGroq(temperature=0, model_name="llama3-70b-8192", groq_api_key = groq_api_key)

In [26]:
def retrieve_answer(query):
    retriever = db.as_retriever()
    docs = retriever.invoke(query, search_kwargs={"k": 3})
    context = '\n\n'.join([doc.page_content for doc in docs[:3]])
    answer = llm.predict(text=PROMPT.format_prompt(
            context=context,
            question=query).text)
    print(context)
    return answer

In [27]:
res = retrieve_answer("Какие число подписчиков на Яндекс Плюс?")

(10 542) (2 209) н/прим. (19 644) (23 611) 20%
Плюс
и развлекательн
ые сервисы
Число подписчиков
Яндекс Плюса, млн 19,3 30,4 58% 19,3 30,4 58%
Финансовый прогноз
На сегодняшний день возможность прогнозирования в краткосрочной и среднесрочной перспективе
по-прежнему остаётся ограниченной из-за сохраняющейся высокой степени неопределённости в
отношении геополитических и макроэкономических событий в будущем. Мы планируем продолжать
подробно информировать о результатах компании.
Корпоративная деятельность и события после отчётной даты
• Нидерландская компания Yandex N.V. заключила сделку по продаже бизнеса Яндекса за 475
млрд руб.* консорциуму частных инвесторов (больше подробностей — здесь) во главе с
3
менеджерами Яндекса. Yandex N.V. перестанет быть головной компанией группы. До 31 июля
2024 года она сменит название и прекратит использовать бренды Яндекса.
• Новой головной компанией Яндекса станет Международная компания акционерное общество

спроса, а также повышением операционной эффек

In [28]:
res

'Число подписчиков Яндекс Плюса в четвёртом квартале 2023 года достигло 30,4 миллиона, увеличившись на 58% по сравнению с аналогичным показателем на конец четвёртого квартала 2022 года.'